In [1]:
# importuje niezbędne biblioteki
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import os
import glob

In [2]:
# Tworzymy listę i dwa puste DF
listOfFiles = []
DFL = pd.DataFrame()
DFM = pd.DataFrame() 

In [3]:
# Funkcja pobiera listę ścieżek
def get_filenames():
    Tk().withdraw()
    print("Initializing Dialogue... \nPlease select a file.")
    tk_filenames = filedialog.askdirectory()
    tempDir = tk_filenames
    return tempDir


choosen_dir = get_filenames()
# glob zbiera listę plików 
os.chdir(choosen_dir)
for file in glob.glob("*.csv"):
    listOfFiles.append(file)

# Do wcześniej stworzonych DF dodajemy kolejno pliki w zależności od tego na jaką literę plik się zaczyna.
for file in listOfFiles:
    if file.startswith('L'):
        Table = pd.read_csv(file, sep=',')
        DF1 = pd.DataFrame(Table)
        DFL = DFL.append(DF1, sort=False)
    elif file.startswith('M'):
        Table2 = pd.read_csv(file, sep=',')
        DF2 = pd.DataFrame(Table2)
        DFM = DFM.append(DF2, sort=False)

Initializing Dialogue... 
Please select a file.


In [5]:
# Deklaruje listę kolumn którę będą mi potrzebne
DFL['NMIOT'] = 1
cols1 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','NROJM1','LSUT1', 'NRMAM1', 'NOJMIO']
DFL = DFL[cols1]

In [6]:
# Zmieniam nazwy kolumn tak by odpowiadały kolumnon z DFL
DFM['LSUT1'] = np.nan  
DFM = DFM.rename(columns={'LDUR': 'LDUR1'})
DFM = DFM.rename(columns={'LDOP': 'LDUR'})
DFM = DFM.rename(columns={'LDUR1': 'LDOP'})
DFM = DFM.rename(columns={'RASAOJ': 'LRASAOJ'})
DFM = DFM.rename(columns={'NRO': 'NROJM1'})
DFM = DFM.rename(columns={'LREJ': 'LREJ1'})
DFM = DFM.rename(columns={'LWOJ': 'LWOJ1'})
# Deklaruje listę kolumn którę będą mi potrzebne
cols2 = ['LPROS1', 'LRASA1', 'LDUR', 'LDOP', 'LRASAOJ', 'LIL11', 'LIL21','LWAGA1','M_WAGUR','LDBRAK','NMIOT','LREJ1',
        'LWOJ1', 'LSEK1', 'LSTA1','LSUT1','NROJM1']
DFM = DFM[cols2]

In [7]:
# Wprowadzamy przedział czasowy z jakiego będą pobieranie dane
date1 = input('Date range from YYYY-MM-DD') 
date2 = input('Date range to YYYY-MM-DD')

Date range from YYYY-MM-DD 2020-12-12
Date range to YYYY-MM-DD 2021-12-11


In [8]:
# W DF zostawaimy tylko dane z wcześniej zadeklarowanego przedziału czasowego
DFL = DFL[(DFL['LDOP'] >= date1) & (DFL['LDOP'] <= date2)]
DFM = DFM[(DFM['LDOP'] >= date1) & (DFM['LDOP'] <= date2)]

In [9]:
# Funkcja zamienia format kolumn na format daty
class ToDate():

    def change_to_date(self, a, b):
        a[b] = pd.to_datetime(a[b])


dat = ToDate()
dat.change_to_date(DFL, 'LDOP')
dat.change_to_date(DFM, 'LDOP')
dat.change_to_date(DFL, 'LDUR')
dat.change_to_date(DFM, 'LDUR')
# Okres międzymiotu
DFM['OMM'] = DFM['LDOP'] - DFM['LDUR']
DFM['OMM'] = DFM['OMM'] / np.timedelta64(1, 'D')

In [10]:
# W zmiennej DF3 łączymy poprzednie DF
DF3 = DFL.append(DFM)
DF3 = DF3.rename(columns={'LDUR': 'DURL'})
DF3 = DF3.rename(columns={'LDOP': 'DURM'})

In [11]:
# Wyliczamy ile prosiąt nie dożyło 21 dnia życia
DF3['STRATY'] = DF3['LIL11'] - DF3['LIL21']
# Wyliczamy wartość % strat
DF3['ST21'] = (DF3['STRATY'] / DF3['LIL11']) * 100

In [12]:
DF3['Miot ogółem'] = len(DF3[(DF3.LRASA1 == 10)])

In [13]:
# Funkcja tworzy DF z określonymi już kolumnami
def createDT(name):
    temp = pd.DataFrame(columns = ['Kolejny miot','Rasa','Mioty szt.', 
                                   'Mioty %','Liczba prosiąt urodzonych w miocie',
                                   'Liczba prosiąt w 21 dniu życia','Straty prosiąt do 21 dnia życia',
                                   'Okres miedzymiotu(dni)'],index =[1])
    temp['Rasa'] = name
    return temp
# Funkcja sortuje dane po rasie matki i rasie ojca
def sorted2DT(DT,number1,number2):
    newDT = DT[(DT.LRASA1 == number1) & (DT.NMIOT == number2)]
    return newDT

def months(DT, col, col2):
    newDT = DT[DT[col] != DT[col2]]
    return newDT
# Funkcja wylicza wszystkie niezbędne wartości i przypisuje je do DF
def all_count(DT, new,rasa):
    miot = DT['NMIOT'].unique()
    new['Kolejny miot'] = miot[0]
    new['Mioty szt.'] = len(DT)
    new['Mioty %'] = np.around(len(DT)/len(DF3[(DF3.LRASA1 == rasa)])*100,decimals=2)
    new['Liczba prosiąt urodzonych w miocie'] = np.around(DT['LIL11'].mean(), decimals=2)
    new['Liczba prosiąt w 21 dniu życia'] = np.around(DT['LIL21'].mean(), decimals=2)
    new['Straty prosiąt do 21 dnia życia'] = np.around(DT['ST21'].mean(), decimals=2)
    new['Okres miedzymiotu(dni)'] = np.around(DT['OMM'].mean())

    return new

In [14]:
race_dict = {
    10 : "WBP",
    20 : "PBZ",
    30 : "Złotnicka Biała",
    40 : "Póławska",
    50 : "Złotnicka Pstra",
    60 : "Hampshire",
    70 : "Duroc",
    80 : "Pietrain",
    90 : "Linia 990"}

In [15]:
wbp = createDT('Wbp')
WBP = sorted2DT(DF3,10,1)
all_count(WBP,wbp,10)

,Kolejny miot,Rasa,Mioty szt.,Mioty %,Liczba prosiąt urodzonych w miocie,Liczba prosiąt w 21 dniu życia,Straty prosiąt do 21 dnia życia,Okres miedzymiotu(dni)
1,1,Wbp,539,20.13,11.92,11.2,5.67,NaN


In [16]:
completeDF = pd.DataFrame()
for race in race_dict:
    dokladna_rasa = DF3[(DF3.LRASA1 == race)]
    mioty = dokladna_rasa['NMIOT'].unique()
    for i in mioty:
        aDF = createDT(race_dict[race])
        bDF = sorted2DT(DF3,race,i)
        rdy = all_count(bDF,aDF,race)
        completeDF = pd.concat([completeDF,rdy])

In [17]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
completeDF.set_index(['Rasa','Kolejny miot']).sort_values(by = ['Rasa','Kolejny miot'])

Mioty szt.  Mioty %  \
Rasa            Kolejny miot                        
Duroc           1                    115    22.64   
                2                     91    17.91   
                3                     68    13.39   
                4                     73    14.37   
                5                     57    11.22   
                6                     28     5.51   
                7                     22     4.33   
                8                     12     2.36   
                9                     13     2.56   
                10                    10     1.97   
                11                     3     0.59   
                13                     1     0.20   
                14                     4     0.79   
                15                     7     1.38   
                16                     2     0.39   
                19                     1     0.20   
                20                     1     0.20   
Hampshire       2                      1     4.35   
                3                      5    21.74   
                4                      4    17.39   
                8                      2     8.70   
                9                      2     8.70   
                10                     1     4.35   
                12                     1     4.35   
                13                     1     4.35   
                15                     1     4.35   
                16                     4    17.39   
                17                     1     4.35   
Linia 990       1                    197    19.64   
                2                    137    13.66   
                3                    156    15.55   
                4                    131    13.06   
                5                    106    10.57   
                6                     89     8.87   
                7                     66     6.58   
                8                     57     5.68   
                9                     33     3.29   
                10                    15     1.50   
                11                    10     1.00   
                12                     3     0.30   
                13                     2     0.20   
                14                     1     0.10   
PBZ             1                    848    19.73   
                2                    880    20.47   
                3                    742    17.26   
                4                    563    13.10   
                5                    418     9.73   
                6                    307     7.14   
                7                    182     4.23   
                8                    134     3.12   
                9                     74     1.72   
                10                    61     1.42   
                11                    32     0.74   
                12                    21     0.49   
                13                    12     0.28   
                14                    11     0.26   
                15                     7     0.16   
                16                     6     0.14   
Pietrain        1                     69    17.29   
                2                     64    16.04   
                3                     63    15.79   
                4                     51    12.78   
                5                     44    11.03   
                6                     36     9.02   
                7                     30     7.52   
                8                     15     3.76   
                9                      6     1.50   
                10                     4     1.00   
                11                     5     1.25   
                12                     5     1.25   
                13                     4     1.00   
                14                     2     0.50   
                15                     1     0.25   
Póławska        1                    659    17.89   
  